# RSE Group Leaders Survey 2023

In [13]:
import sys
import importlib
import numpy as np
import pandas as pd
sys.path.append('../src/data/')
import util as util

In [2]:
setup, dset = util.read_setup_and_data()
validated = util.validate_setup_and_data(setup, dset)

if not validated:
    print("ERROR: setup and data files are not valid.")
    raise KeyboardInterrupt

# number of respondents and questions
nrsps_raw = dset.shape[0]
nqsts = dset.shape[1]

# replace dset columns with setup keys
dset.columns = setup.keys()

# remove respondents who did not agree to share data
dset = dset[dset.q1 == "Yes"]
nrsps = dset.shape[0]

# process checkbox responses
dset = util.process_checkboxes(dset, setup)

# save the processed dataset
util.save_data(dset)

# print out some information
print(f"Questions  : {nqsts}")
print(f"Respondents - all           : {nrsps_raw}")
print(f"Respondents - agreed sharing: {nrsps}")


Questions  : 20
Respondents - all           : 36
Respondents - agreed sharing: 36


## Overview of the answers

In [17]:
for qst in setup.keys():
    if setup[qst]['type'] == 'checkbox':
        print(f"\n{qst.upper()}: {setup[qst]['text']}\n")
        util.display_checkbox_stats(dset, setup, qst)


Q4: Is your group:

                                                                                      count  percentage
The main RSE Group for the university/research organisation                              26          72
An RSE Group for a specific subset/group within the university/research organisation      8          22
An RSE Group for multiple universities/research organisations                             4          11

Q7: Where is your group based?

                                                    count  percentage
IT/Research IT                                         14          39
Other                                                   9          25
Academic Department - supporting whole Institution      5          14
Academic Department - supporting a specific domain      4          11
Independent Institute/Facility                          4          11

Q15: Please rate the level of demand for your Group's services

                                             